In [ ]:
'''
Task 1
Fetch MQTTset from Kaggle
'''
import os
from kaggle.api.kaggle_api_extended import KaggleApi

# Authenticate using Kaggle API
api = KaggleApi()
api.authenticate()

# Download the dataset
dataset = 'cnrieiit/mqttset'
api.dataset_download_files(dataset, path='.', unzip=True)

In [1]:
'''
Task 1
Setup SparkSession and test connection to Postgres
'''
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import lit 


# Define dataset, spark constants
APPNAME = 'Roberts_Systool_Project'
MASTER = 'local[3]'
JDBC_JAR_PATH = f"/home/jrob/.cache/pypoetry/virtualenvs/systems-toolchains-C8y3MhrL-py3.10/lib/python3.10/site-packages/pyspark/jars/postgresql-42.7.4.jar"
JDBC_URL = "jdbc:postgresql://localhost:5432/postgres"
DB_PROPERTIES = {
    "user": "postgres",
    "password": "postgres_pw",
    "driver": "org.postgresql.Driver", 
    "batchsize": "10000" # trying to optimize the write and seems to have cut the time in half
}

# Initialize Spark session with PostgreSQL JDBC driver
spark = SparkSession.builder \
    .appName(APPNAME) \
    .master(MASTER)\
    .config('spark.driver.host','127.0.0.1')\
    .config("spark.jars", JDBC_JAR_PATH) \
    .config("spark.driver.memory", "4g") \
    .config("spark.executor.memory", "2g") \
    .getOrCreate()

# df = spark.read.jdbc(url=JDBC_URL, table="mqtt", properties=DB_PROPERTIES)
# df.show()

your 131072x1 screen size is bogus. expect trouble
24/10/12 16:41:51 WARN Utils: Your hostname, Jonathan-Laptop resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
24/10/12 16:41:51 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
24/10/12 16:41:52 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [2]:
'''
Task 1
Ingest data with Spark and write to Postgres

Creates a dictionary for data files and associated data category,
then loops over both values using .items() method

In the loop, it dynamically changes the data category and write mode
based on where the code executes in the loop. First loop overwrites
to create table and schema if not already exists. Second loop appends

Trying optimizations for fun, repartitioning with the rule of thumb of
2x the number of cores and using 3 of the 4 cores while leaving 1 for
other processes increased speed by about 55-60% from 12mins to about
5.75mins. Including batchsize of 10,000 also just about cut the time
in half from there down to right at 2 minutes and 45 seconds.
'''
import os


COL_NAMES = [
    'tcp_flags', 'tcp_time_delta', 'tcp_len', 'mqtt_conack_flags', 
    'mqtt_conack_flags_reserved', 'mqtt_conack_flags_sp', 'mqtt_conack_val', 
    'mqtt_conflag_cleansess', 'mqtt_conflag_passwd', 'mqtt_conflag_qos', 
    'mqtt_conflag_reserved', 'mqtt_conflag_retain', 'mqtt_conflag_uname', 
    'mqtt_conflag_willflag', 'mqtt_conflags', 'mqtt_dupflag', 'mqtt_hdrflags', 
    'mqtt_kalive', 'mqtt_len', 'mqtt_msg', 'mqtt_msgid', 'mqtt_msgtype', 
    'mqtt_proto_len', 'mqtt_protoname', 'mqtt_qos', 'mqtt_retain', 
    'mqtt_sub_qos', 'mqtt_suback_qos', 'mqtt_ver', 'mqtt_willmsg', 
    'mqtt_willmsg_len', 'mqtt_willtopic', 'mqtt_willtopic_len', 'target'
]
DATA_FOLDER = "./data/FINAL_CSV" # won't change due to mqttset zip

data_files = {
    'train70.csv': 'train',
    'test30.csv': 'test'
}
overwrite = True
for file, set_type_value in data_files.items():
    file_path = os.path.join(DATA_FOLDER, file)
    
    # Read CSV file
    df = spark.read.csv(file_path, header=False, inferSchema=True).toDF(*COL_NAMES)
    
    # Add 'set_type' column and dataset value of 'train' or 'test' based on place in dictionary loop
    df = df.withColumn('set_type', lit(set_type_value))
    
    df.repartition(6)
    # Show first 5 rows for verification
    df.show(5)
    
    # Overwrite on the first loop and append on the rest
    write_mode = 'overwrite' if overwrite else 'append'
    
    # Write data to Postgres
    df.write.jdbc(url=JDBC_URL, 
                  table="mqtt", 
                  mode=write_mode, 
                  properties=DB_PROPERTIES,
                  )
    
    # Set overwrite to False after the first write
    if overwrite:
        overwrite = False

24/10/12 16:42:17 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+----------+--------------+-------+-----------------+--------------------------+--------------------+---------------+----------------------+-------------------+----------------+---------------------+-------------------+------------------+---------------------+-------------+------------+-------------+-----------+--------+--------+----------+------------+--------------+--------------+--------+-----------+------------+---------------+--------+------------+----------------+--------------+------------------+----------+--------+
| tcp_flags|tcp_time_delta|tcp_len|mqtt_conack_flags|mqtt_conack_flags_reserved|mqtt_conack_flags_sp|mqtt_conack_val|mqtt_conflag_cleansess|mqtt_conflag_passwd|mqtt_conflag_qos|mqtt_conflag_reserved|mqtt_conflag_retain|mqtt_conflag_uname|mqtt_conflag_willflag|mqtt_conflags|mqtt_dupflag|mqtt_hdrflags|mqtt_kalive|mqtt_len|mqtt_msg|mqtt_msgid|mqtt_msgtype|mqtt_proto_len|mqtt_protoname|mqtt_qos|mqtt_retain|mqtt_sub_qos|mqtt_suback_qos|mqtt_ver|mqtt_willmsg|mqtt_willmsg_l

+----------+--------------+-------+-----------------+--------------------------+--------------------+---------------+----------------------+-------------------+----------------+---------------------+-------------------+------------------+---------------------+-------------+------------+-------------+-----------+--------+--------+----------+------------+--------------+--------------+--------+-----------+------------+---------------+--------+------------+----------------+--------------+------------------+----------+--------+
| tcp_flags|tcp_time_delta|tcp_len|mqtt_conack_flags|mqtt_conack_flags_reserved|mqtt_conack_flags_sp|mqtt_conack_val|mqtt_conflag_cleansess|mqtt_conflag_passwd|mqtt_conflag_qos|mqtt_conflag_reserved|mqtt_conflag_retain|mqtt_conflag_uname|mqtt_conflag_willflag|mqtt_conflags|mqtt_dupflag|mqtt_hdrflags|mqtt_kalive|mqtt_len|mqtt_msg|mqtt_msgid|mqtt_msgtype|mqtt_proto_len|mqtt_protoname|mqtt_qos|mqtt_retain|mqtt_sub_qos|mqtt_suback_qos|mqtt_ver|mqtt_willmsg|mqtt_willmsg_l

In [ ]:
spark.stop()

In [3]:
'''
Task 2
Ingest Data from Postgres into Spark for Data Analysis
'''
num_partitions = 6 # Adjust based on your environment
mqtt_df = spark.read.jdbc(url=JDBC_URL,
                          table="mqtt",
                          properties=DB_PROPERTIES,
                          numPartitions=num_partitions,
                          )


In [3]:
# Number of rows
num_rows = mqtt_df.count()

# Number of columns
num_cols = len(mqtt_df.columns)

print(f"Shape: {num_rows}, {num_cols}")

Shape: 12081189, 35


In [4]:
mqtt_df.printSchema()

# print target col values
# mqtt_df.select("target").distinct().show(truncate=False)

root
 |-- tcp_flags: string (nullable = true)
 |-- tcp_time_delta: double (nullable = true)
 |-- tcp_len: integer (nullable = true)
 |-- mqtt_conack_flags: string (nullable = true)
 |-- mqtt_conack_flags_reserved: double (nullable = true)
 |-- mqtt_conack_flags_sp: double (nullable = true)
 |-- mqtt_conack_val: double (nullable = true)
 |-- mqtt_conflag_cleansess: double (nullable = true)
 |-- mqtt_conflag_passwd: double (nullable = true)
 |-- mqtt_conflag_qos: double (nullable = true)
 |-- mqtt_conflag_reserved: double (nullable = true)
 |-- mqtt_conflag_retain: double (nullable = true)
 |-- mqtt_conflag_uname: double (nullable = true)
 |-- mqtt_conflag_willflag: double (nullable = true)
 |-- mqtt_conflags: string (nullable = true)
 |-- mqtt_dupflag: double (nullable = true)
 |-- mqtt_hdrflags: string (nullable = true)
 |-- mqtt_kalive: double (nullable = true)
 |-- mqtt_len: double (nullable = true)
 |-- mqtt_msg: string (nullable = true)
 |-- mqtt_msgid: double (nullable = true)
 |-

In [12]:
'''
Task 2 
1. Find the average length of an MQTT message captured in
the training dataset for a given target X?

Also trying to use best practices in defining functions for employers
Most analytics will be on the test set when possible to limit processing
when possible and should be large enough, but data engineering will require the whole dataset
'''
from pyspark.sql.functions import col, avg, length


def get_average_len(df, set='test', target=None):
    '''
    Function filters the dataset as much as possible using input parameters
    then aggregates the average mqtt message length and returns that integer value

    :param set: 'train' or 'test' in mqtt_df
    :param target: 'slowite', 'bruteforce', 'flood', 'malformed', 'dos', 'legitimate'
    
    return: int
    '''
    if set not in ('test', 'train'):
        raise ValueError("set parameter must be 'train' or 'test'")
    elif target not in ('slowite', 'bruteforce', 'flood', 'malformed', 'dos', 'legitimate'):
        raise ValueError("target parameter must be one of these: 'slowite', 'bruteforce', 'flood', 'malformed', 'dos', 'legitimate'")
    
    '''
    I think it would be get to .cache() and unpersist(), but I'm afraid I'll break local memory
    so I'll also filter and select to make dataframe as small as possible beforehand
    '''
    len_df = df.filter((col('set_type') == set) & (col('target') == target)).select('mqtt_msg').cache()
    
    # get the right column, then len of values in column, then average, all in an aggregate Spark action method, and .agg() requires .alias()
    avg_len = len_df.agg(avg(length(col('mqtt_msg'))).alias('average_length')).collect()
    len_df.unpersist()
    return avg_len[0]['average_length']

print(f"Average MQTT message length is: {get_average_len(mqtt_df, target='dos'):.2f}")

Average MQTT message length is: 85.53


In [14]:
'''
Task 2 
2. For each target value, 
1) what is the average length of TCP messages and 
2) what is the most popular header flags’ code (mqtt.hdrflags)?

Given the task, it seems no other parameters are necessary
'''
from pyspark.sql.functions import col, avg, length


def get_all_avg_tcp_len(df):
    '''
    Function loops over each unique value in the target column
    to add the average tcp message length to a dictionary and
    returns a dictionary with the avg tcp message length for
    each target value and that target's most popular hdrflag
    placed in a dictionary of tuples where [0]=tcp_len and
    [1]=popular_header 

    Example: {'target':(50, 'hdrflag_val')}

    return: dict
    '''
    answer = {}
    targets = [row['target'] for row in mqtt_df.select('target').distinct().collect()]
    
    # filters for tcp messages and pulls only necessary columns
    tcp_df = df.filter((col('tcp_len') != 0)).select('target', 'tcp_len', 'mqtt_hdrflags').cache()
    
    # Loop over every unique value in target column to get avg tcp_len and top mqtt_hdrflag value
    for target in targets:
        # Filter by target, then get average length
        target_df = tcp_df.filter(col('target') == target).cache()
                         
        tcp_len = target_df.agg(avg(length(col('tcp_len'))).alias('average_length')).collect()
        pop_hdr = target_df.groupBy('mqtt_hdrflags').count().orderBy(col('count').desc()).first()
        
        target_df.unpersist()
        answer[target] = (tcp_len[0]['average_length'], pop_hdr['mqtt_hdrflags'])
    # get the right column, then len of values in column, then average, all in an aggregate Spark action method, and .agg() requires .alias()
    tcp_df.unpersist()
    return answer

ans = get_all_avg_tcp_len(mqtt_df)
keys = ans.keys()
for key in keys:
    print(f"Average TCP message length for target value {key} is: {ans[key][0]:.2f} \n and the most popular header value for {key} is: {ans[key][1]}")

24/10/12 17:34:25 WARN CacheManager: Asked to cache already cached data.        
24/10/12 17:34:25 WARN CacheManager: Asked to cache already cached data.


Average TCP message length for target value slowite is: 1.34 
 and the most popular header value for slowite is: 0
Average TCP message length for target value bruteforce is: 1.51 
 and the most popular header value for bruteforce is: 0x00000010
Average TCP message length for target value flood is: 4.82 
 and the most popular header value for flood is: 0x00000030
Average TCP message length for target value malformed is: 1.79 
 and the most popular header value for malformed is: 0x00000030
Average TCP message length for target value dos is: 2.42 
 and the most popular header value for dos is: 0x00000040
Average TCP message length for target value legitimate is: 1.97 
 and the most popular header value for legitimate is: 0x00000030


In [17]:
'''
3. What is the most frequent X TCP flags for traffic with TCP time delta that is smaller than or equal to Y.
- [] X represents a positive integer. Handle scenarios where a user may send
negative values to your function.
- [] Y represents a float value between 0.0 and 5.0.
- [] Make sure to handle this scenario as well: if the user requests 5 most frequent TCP flags but there are 3 Flags that share the same count at rank number 5, please include all of them in your output.

Believing that most frequent X means top X(int) TCP Flags and ranks were not asked for, funtion returns a list of flags
'''

def get_most_frequent_flags(df, X: int, Y: float):
    '''
    Function filters for tcp_flags and tcp_time_delta to minimize data
    for processing. Handles errors by raisng ValueErrors and includes rank
    ties as they appear.

    return: list
    '''
    if 0 >= X:
        raise ValueError("X must be a positive integer.")
    if not (5.0 >= Y >= 0.0):
        raise ValueError("Y must be a float between 0.0 and 5.0.")
    
    # Filter by tcp_time_delta less than Y, then count values in tcp_flags and collect rows
    flags_df = df.filter(col('tcp_time_delta') <= Y).select('tcp_flags')
    desc_flags = flags_df.groupBy('tcp_flags').count().orderBy(col('count').desc()).collect()

    top_flags = []
    for i, row in enumerate(desc_flags):
        # Continue until X int is reached or if the previous row count equals the current row counts to add ties
        if X > i or row['count'] == desc_flags[i - 1]['count']:
            top_flags.append(row['tcp_flags'])
    return top_flags

flags = get_most_frequent_flags(mqtt_df, X=4, Y=.003)
print('The most frequenXt flags for tcp_time_delta < Y are:')
for flag in flags:
    print(f'{flag}') 

The most frequenXt flags for tcp_time_delta < Y are:

0x00000010
0x00000018
0x00000011
0x00000002
